#### Usosweb documents scraping

In [4]:
from USOSDataLoader import USOSDataLoader

loader = USOSDataLoader()
documents = loader.get_documents()

Loading web data...


Preprocessing documents...: 100%|██████████| 81/81 [00:00<00:00, 3710.15it/s]


#### Knowledge base generation

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [25]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os

embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v3",
                                   model_kwargs={"trust_remote_code": True},
                                   encode_kwargs={"task": "retrieval.query"})

pc = Pinecone(os.environ.get("PINECONE_API_KEY"))

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [26]:
import time
from pinecone import ServerlessSpec

index_name = "usos-bot"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [27]:
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
uuids = [str(uuid4()) for _ in range(len(splits))]
vectorstore.add_documents(documents=splits, ids=uuids)

['f9024215-646b-4f00-aa8c-e096e4332428',
 'bcb2606a-57bd-4ce7-aef1-a698f4f78943',
 'c1d77680-0812-4c8c-8151-650ca06b5a78',
 '923dc4af-7b53-4fc0-bd3c-1a229922a55c',
 'a630b58b-16ed-4062-9975-e50f7bad3257',
 'b3010264-0e48-4b62-b983-6440799da90f',
 'cf01e485-b732-4c2d-9442-2d149b3ca18d',
 'e30df47e-aa03-425b-8483-b950f90cf2b3',
 '04fc0715-3046-46cc-acab-e1931d011d02',
 'dca7447a-def0-486d-b5cd-897152998ffd',
 '2dfa247f-feaf-48ad-8d50-5cb8c69b9945',
 'd5841b8a-dc41-4dfb-9d08-aa3367d3c222',
 '0f97303d-7118-4a24-9d1c-544c231db4ce',
 '20f16703-c12e-483c-bc07-ff5c55a191e3',
 '4a9a6a31-8094-42ee-ad58-3dd4c2d0701a',
 '60223a7c-be3f-4310-9b23-122b5796cb28',
 'c259c42f-6ea1-42f3-95a3-e85fd2f7c3e1',
 '9afa880e-bd7f-41d2-a10d-d4a5d4401e85',
 '865cf58a-783b-4eae-8ded-316fa8ee97e3',
 'ff3dbea8-264a-47e9-b7da-7ed385f988ca',
 '7ede6405-f9c2-4084-b1ba-98c4787efbff',
 'b8891129-408c-458c-8e60-96c4f09ba5f2',
 '59cd3f9c-6a3a-475b-b996-ba2b845d57e4',
 '850c0243-9a6b-4bfa-80fd-9c760ee17152',
 '522f634d-f3fc-